In [5]:
import argparse
import torch
import os
import json
from tqdm import tqdm
import shortuuid

from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, process_images, get_model_name_from_path
from torch.utils.data import Dataset, DataLoader
import math


def split_list(lst, n):
    """Split a list into n (roughly) equal-sized chunks"""
    chunk_size = math.ceil(len(lst) / n)  # integer division
    return [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]


def get_chunk(lst, n, k):
    chunks = split_list(lst, n)
    return chunks[k]


# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data_path, tokenizer, model_config):
        self.list_data_dict = json.load(open(data_path, "r"))
        self.tokenizer = tokenizer
        self.model_config = model_config

    def __getitem__(self, index):
        sources = self.list_data_dict[index]
        qs = sources["conversations"][0]["value"]
        # if self.model_config.mm_use_im_start_end:
        #     qs = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + qs
        # else:
        #     qs = DEFAULT_IMAGE_TOKEN + '\n' + qs

        conv = conv_templates[args.conv_mode].copy()
        conv.append_message(conv.roles[0], qs)
        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()
        #print(prompt)
        image_tensor=torch.Tensor(sources['tensor'])
        input_ids = tokenizer_image_token(prompt, self.tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt')

        return input_ids, image_tensor

    def __len__(self):
        return len(self.list_data_dict)


def collate_fn(batch):
    input_ids, image_tensors = zip(*batch)
    input_ids = torch.stack(input_ids, dim=0)
    image_tensors = torch.stack(image_tensors, dim=0)
    return input_ids, image_tensors


# DataLoader
def create_data_loader(data_path, tokenizer, model_config, batch_size=1, num_workers=4):
    assert batch_size == 1, "batch_size must be 1"
    dataset = CustomDataset(data_path, tokenizer, model_config)
    data_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, collate_fn=collate_fn)
    return data_loader


# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--model-path", type=str, default="facebook/opt-350m")
#     parser.add_argument("--model-base", type=str, default=None)
#     parser.add_argument("--data-path", type=str, default="tables/question.jsonl")
#     parser.add_argument("--answers-file", type=str, default="answer.jsonl")
#     parser.add_argument("--conv-mode", type=str, default="llava_v1")
#     parser.add_argument("--num-chunks", type=int, default=1)
#     parser.add_argument("--chunk-idx", type=int, default=0)
#     parser.add_argument("--temperature", type=float, default=0.2)
#     parser.add_argument("--top_p", type=float, default=None)
#     parser.add_argument("--num_beams", type=int, default=1)
#     parser.add_argument("--max_new_tokens", type=int, default=128)
#     args = parser.parse_args()

#     eval_model(args)

def generate_image(input_ids,model):
    output_img=[]
    inputs_embeds=model.get_model().embed_tokens(input_ids) #1, seq_le, 4096
    with torch.inference_mode():
        for i in range(6):
            outputs = model.model(
                input_ids=None,
                attention_mask=None,
                position_ids=None,
                past_key_values=None,
                inputs_embeds=inputs_embeds,
            )
            hidden_states = outputs[0]
            img = model.get_model().mm_projector_inverse(hidden_states[:,-1,:])
            output_img.append(img)
            new_embed=model.get_model().mm_projector(img)
            new_embed=new_embed.unsqueeze(1).to(inputs_embeds.device)
            inputs_embeds=torch.cat([inputs_embeds,new_embed],dim=1)
            
    return output_img

def generate_image_hidden(input_ids,model):
    output_img=[]
    output_embed=[]
    inputs_embeds=model.get_model().embed_tokens(input_ids) #1, seq_le, 4096
    with torch.inference_mode():
        for i in range(6):
            outputs = model.model(
                input_ids=None,
                attention_mask=None,
                position_ids=None,
                past_key_values=None,
                inputs_embeds=inputs_embeds,
            )
            hidden_states = outputs[0]
            output_embed.append(hidden_states[:,-1,:])
            img = model.get_model().mm_projector_inverse(hidden_states[:,-1,:])
            output_img.append(img)
            new_embed=model.get_model().mm_projector(img)
            new_embed=new_embed.unsqueeze(1).to(inputs_embeds.device)
            inputs_embeds=torch.cat([inputs_embeds,new_embed],dim=1)
    output_embed=torch.stack(output_embed,dim=1) #1,6,4096
            
    return output_img,output_embed

In [7]:
#load trained model
ckp_list=[390]
model_name='llava-v1.5-7b-2-lora'
model_list=[f'/datadrive_a/jihai/LLaVA/scripts/v1_5/checkpoints/{model_name}/checkpoint-{i}' for i in ckp_list]
args = type('Args', (), {
    "model_path": model_list[0],
    "model_base": '/datadrive_a/tmp/vicuna-7b-v1.5/vicuna-7b-v1.5',
    "data_path": '/datadrive_a/jihai/data/multimodalout/dummy_data_inverse_u_eval.json',
    "answers_file": f"./answer/answer-{model_name}-inverse.jsonl",
    "conv_mode": "llava_v1",
    "num_chunks": 1,
    "chunk_idx": 0,
    "temperature": 0,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 128
})()
print(args.model_path)
disable_torch_init()
model_path = os.path.expanduser(args.model_path)
print(model_path)
model_name = get_model_name_from_path(model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, args.model_base, model_name)



answers_file = args.answers_file
os.makedirs(os.path.dirname(answers_file), exist_ok=True)
ans_file = open(answers_file, "w")
if 'plain' in model_name and 'finetune' not in model_name.lower() and 'mmtag' not in args.conv_mode:
    args.conv_mode = args.conv_mode + '_mmtag'
    print(f'It seems that this is a plain model, but it is not using a mmtag prompt, auto switching to {args.conv_mode}.')

data_loader = create_data_loader(args.data_path, tokenizer, model.config)
list_data_dict = json.load(open(args.data_path, "r"))



/datadrive_a/jihai/LLaVA/scripts/v1_5/checkpoints/llava-v1.5-7b-2-lora/checkpoint-390
/datadrive_a/jihai/LLaVA/scripts/v1_5/checkpoints/llava-v1.5-7b-2-lora/checkpoint-390
Loading LLaVA from base model...


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.09s/it]
Some weights of LlavaLlamaForCausalLM_ImgGen were not initialized from the model checkpoint at /datadrive_a/tmp/vicuna-7b-v1.5/vicuna-7b-v1.5 and are newly initialized: ['model.mm_projector.bias', 'model.mm_projector.weight', 'model.mm_projector_inverse.bias', 'model.mm_projector_inverse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading additional LLaVA weights...
Parameter containing:
tensor([[-0.0075, -0.0016, -0.0100,  ..., -0.0041,  0.0129,  0.0048],
        [ 0.0124,  0.0017,  0.0127,  ...,  0.0151, -0.0141,  0.0030],
        [ 0.0082, -0.0147, -0.0095,  ...,  0.0035, -0.0144, -0.0128],
        ...,
        [-0.0047, -0.0109,  0.0116,  ...,  0.0026, -0.0015,  0.0102],
        [ 0.0009, -0.0130, -0.0011,  ...,  0.0105,  0.0043, -0.0028],
        [-0.0143,  0.0001,  0.0108,  ..., -0.0028,  0.0132, -0.0126]],
       device='cuda:3', dtype=torch.float16, requires_grad=True)
Parameter containing:
tensor([[ 0.3440, -0.3057,  0.2678,  ..., -0.3342,  0.0781, -0.3284],
        [ 0.1821,  0.2266, -0.3391,  ..., -0.0566,  0.1644,  0.3623],
        [-0.0182,  0.2235,  0.3613,  ..., -0.1952, -0.2522, -0.2957],
        ...,
        [-0.3052,  0.0137, -0.2708,  ..., -0.3171, -0.0924,  0.0781],
        [-0.1660, -0.3328,  0.3513,  ...,  0.1301, -0.0415, -0.3669],
        [-0.1798,  0.1610,  0.2468,  ..., -0.2338,  0.1555

In [8]:


count=0

# 将 zip 对象转换为迭代器
data_iter = iter(zip(data_loader, list_data_dict))

# 使用 tqdm 包裹迭代器以显示进度条
progress_bar = tqdm(total=len(list_data_dict))

while True:
    try:
         # 手动获取下一个值
        (input_ids, image_tensor), line = next(data_iter)
        
        # 在这里可以随时控制 input_ids, image_tensor 和 line 的处理逻辑
#for (input_ids, image_tensor), line in tqdm(zip(data_loader, list_data_dict), total=len(list_data_dict)):
        # count+=1
        # if count==500: break

        cur_prompt = line["conversations"][0]["value"]
        groun_truth=line["conversations"][1]["value"]
        groun_truth_img_tensor=line["tensor"]
        input_ids = input_ids.to(device='cuda', non_blocking=True)
        image_tensor=image_tensor.to(dtype=torch.float16, device='cuda', non_blocking=True)
        with torch.inference_mode():
            outputs = model.generate(
                input_ids,
                images=image_tensor,
                do_sample=True if args.temperature > 0 else False,
                temperature=args.temperature,
                top_p=args.top_p,
                num_beams=args.num_beams,
                max_new_tokens=args.max_new_tokens,
                use_cache=True)
        output_ids=outputs['generated_tokens']
        outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=False)[0].strip()
        #print(outputs)

        img_indicator = torch.tensor([529,  3027, 29958])
        id_seq = output_ids[0].cpu()

        # 子序列长度
        sub_seq_len = len(img_indicator)

        # 滑动窗口查找子序列
        start_idx = -1
        for i in range(id_seq.size(0) - sub_seq_len + 1):
            if torch.equal(id_seq[i:i + sub_seq_len], img_indicator):
                start_idx = i
                break
        img=None
        img_embed=None
        if start_idx != -1:
            output_ids=output_ids[:,1:start_idx+3]
            input_ids=torch.cat((input_ids, output_ids), dim=1)
            img,img_embed=generate_image_hidden(input_ids,model)
            img=torch.stack(img,dim=0).squeeze().cpu().tolist()
            ans_file.write(json.dumps({"prompt": cur_prompt,
                                "groun_truth": groun_truth,
                                "answer": outputs,
                                "groun_truth_img_tensor": groun_truth_img_tensor,
                                "output_img_tensor": img,
                                "model_id": model_name,
                                "metadata": {}}) + "\n")
            

        (input_ids, image_tensor), line = next(data_iter)
        cur_prompt = line["conversations"][0]["value"]
        groun_truth=line["conversations"][1]["value"]
        groun_truth_img_tensor=line["tensor"]
        input_ids = input_ids.to(device='cuda', non_blocking=True)
        image_tensor=image_tensor.to(dtype=torch.float16, device='cuda', non_blocking=True)
        with torch.inference_mode():
            outputs = model.generate(
                input_ids,
                images=image_tensor,
                do_sample=True if args.temperature > 0 else False,
                temperature=args.temperature,
                top_p=args.top_p,
                num_beams=args.num_beams,
                max_new_tokens=args.max_new_tokens,
                use_cache=True,
                input_img_features=img_embed.squeeze())
        output_ids=outputs['generated_tokens']
        outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=False)[0].strip()
        ans_file.write(json.dumps({"prompt": cur_prompt,
                                    "groun_truth": groun_truth,
                                    "answer": outputs,
                                    "groun_truth_img_tensor": groun_truth_img_tensor,
                                    "output_img_tensor": img,
                                    "model_id": model_name,
                                    "metadata": {}}) + "\n")
        progress_bar.update(1)

    except StopIteration:
        # 当迭代器耗尽时退出循环
        break
ans_file.close() 
progress_bar.close()

 50%|█████     | 100/200 [03:25<03:25,  2.06s/it]


In [ ]:
#load trained model
ckp_list=[390]
model_name='llava-v1.5-7b-mix-u-odd-lora'
model_list=[f'/datadrive_a/jihai/LLaVA/scripts/v1_5/checkpoints/{model_name}/checkpoint-{i}' for i in ckp_list]
for k in range(len(model_list)):
    args = type('Args', (), {
        "model_path": model_list[k],
        "model_base": '/datadrive_a/tmp/vicuna-7b-v1.5/vicuna-7b-v1.5',
        "data_path": '/datadrive_a/jihai/data/multimodalout/dummy_data_u_even_eval.json',
        "answers_file": f"./answer/answer-{model_name}-{k}.jsonl",
        "conv_mode": "llava_v1",
        "num_chunks": 1,
        "chunk_idx": 0,
        "temperature": 0,
        "top_p": None,
        "num_beams": 1,
        "max_new_tokens": 128
    })()
    print(args.model_path)
    disable_torch_init()
    model_path = os.path.expanduser(args.model_path)
    print(model_path)
    model_name = get_model_name_from_path(model_path)
    tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, args.model_base, model_name)



    answers_file = args.answers_file
    os.makedirs(os.path.dirname(answers_file), exist_ok=True)
    ans_file = open(answers_file, "w")
    if 'plain' in model_name and 'finetune' not in model_name.lower() and 'mmtag' not in args.conv_mode:
        args.conv_mode = args.conv_mode + '_mmtag'
        print(f'It seems that this is a plain model, but it is not using a mmtag prompt, auto switching to {args.conv_mode}.')

    data_loader = create_data_loader(args.data_path, tokenizer, model.config)
    list_data_dict = json.load(open(args.data_path, "r"))


    count=0
    for (input_ids, image_tensor), line in tqdm(zip(data_loader, list_data_dict), total=len(list_data_dict)):
        count+=1
        if count==500: break
    
        cur_prompt = line["conversations"][0]["value"]
        groun_truth=line["conversations"][1]["value"]
        groun_truth_img_tensor=line["tensor"]
        input_ids = input_ids.to(device='cuda', non_blocking=True)
        image_tensor=image_tensor.to(dtype=torch.float16, device='cuda', non_blocking=True)
        with torch.inference_mode():
            outputs = model.generate(
                input_ids,
                images=image_tensor,
                do_sample=True if args.temperature > 0 else False,
                temperature=args.temperature,
                top_p=args.top_p,
                num_beams=args.num_beams,
                max_new_tokens=args.max_new_tokens,
                use_cache=True)
        output_ids=outputs['generated_tokens']
        outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=False)[0].strip()
        #print(outputs)

        img_indicator = torch.tensor([529,  3027, 29958])
        id_seq = output_ids[0].cpu()

        # 子序列长度
        sub_seq_len = len(img_indicator)

        # 滑动窗口查找子序列
        start_idx = -1
        for i in range(id_seq.size(0) - sub_seq_len + 1):
            if torch.equal(id_seq[i:i + sub_seq_len], img_indicator):
                start_idx = i
                break
        img=None
        if start_idx != -1:
            output_ids=output_ids[:,1:start_idx+3]
            input_ids=torch.cat((input_ids, output_ids), dim=1)
            img=generate_image(input_ids,model)
            
            img=torch.stack(img,dim=0).squeeze().cpu().tolist()
        ans_file.write(json.dumps({"prompt": cur_prompt,
                                    "groun_truth": groun_truth,
                                    "answer": outputs,
                                    "groun_truth_img_tensor": groun_truth_img_tensor,
                                    "output_img_tensor": img,
                                    "model_id": model_name,
                                    "metadata": {}}) + "\n")
        #outputs = tokenizer.batch_decode(input_ids, skip_special_tokens=False)[0].strip()
    
    print(ans_file)
    ans_file.close() 
